In [1]:
from __future__ import print_function
from socket import timeout
from bs4 import BeautifulSoup
import urllib.request
import re
import html.parser
import os
from pip._vendor.distlib.compat import raw_input

Загружаем эссе из письменной части IELTS

In [2]:
url = 'https://www.ielts-exam.net/ielts_writing_samples_task_2/1129/'
i = 0
texts_list = []
while i <50:
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = urllib.request.Request(url, headers=hdr)
    sauce = urllib.request.urlopen(req,timeout=10).read()
    soup = BeautifulSoup(sauce, 'html.parser')
    par_list = [i.text for i in soup.find_all('p')]
    if 'Write at least 250 words.' in par_list:
        par_list = par_list[par_list.index('Write at least 250 words.')+1:-4]
    else:   
            par_list = par_list[par_list.index('You should write at least 250 words.')+1:-4]
    texts_list.append((' '.join(par_list),url))
    url = soup.find('li',class_='next').a.get('href')
    i+=1 


In [3]:
url = 'https://www.ielts-exam.net/ielts_writing_samples_task_2/252/'
i = 0
while i <50:
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = urllib.request.Request(url, headers=hdr)
    sauce = urllib.request.urlopen(req,timeout=10).read()
    soup = BeautifulSoup(sauce, 'html.parser')
    par_list = [i.text for i in soup.find_all('p')]
    try: par_list = par_list[par_list.index('model answer:')+1:-4]
    except:
        print(url)
        url = soup.find('li',class_='next').a.get('href')
        continue
    texts_list.append((' '.join(par_list),url))
    url = soup.find('li',class_='next').a.get('href')
    i+=1 

https://www.ielts-exam.net/ielts_writing_samples_task_2/150/
https://www.ielts-exam.net/ielts_writing_samples_task_2/143/
https://www.ielts-exam.net/ielts_writing_samples_task_2/61/


In [4]:
import spacy
nlp = spacy.load("en_core_web_sm")
corp_texts = {}
corp_sents = {}
corp_words = {}
for i, text in enumerate(texts_list):
    corp_texts[str(i)]= {}
    corp_texts[str(i)]['text'] = re.sub(r'\\[rn]','', text[0])
    corp_texts[str(i)]['url'] = text[1]
    doc = nlp(text[0])
    for j, s in enumerate(doc.sents):
        sent_id= str(i)+'_'+str(j)
        corp_sents[sent_id] = s.text
        for t in s:
            if not t.text=='.':
                word = t.text.lower()
                if word in corp_words.keys():
                    if not sent_id in corp_words[word]['sent_ids']:
                        corp_words[word]['sent_ids'].append(sent_id)
                else:
                    corp_words[word]={}
                    corp_words[word]['lemma']= t.lemma_
                    corp_words[word]['pos']=  t.pos_
                    corp_words[word]['sent_ids']=[sent_id]

NameError: name 'corp_texts' is not defined

In [7]:
def determ_ent(en):
    if not re.match(r'"[a-z]*"',en) is None:
        return((en.replace('"',''),'lemma'))
    elif not re.match(r'[A-Z]+',en) is None:
        return(en,'pos')
    elif not re.match(r'[a-z]+',en) is None:
        return(en,'word')
    else:
        raise ValueError('Wrong search input')
def search(for_search):
    if for_search[-1]=='word':
        if for_search[0] in corp_words.keys():
            ids = corp_words[for_search[0]]['sent_ids']
            for _id in ids:
                _id.split('_')[0]
                print(corp_sents[_id]+' '+corp_texts[_id.split('_')[0]]['url']+'\n')
    else:
        for w in corp_words.items():
            if w[1][for_search[1]] ==for_search[0]:
                print('\n\n'+w[0]+'\n\n')
                ids = w[1]['sent_ids']
                for _id in ids:
                    _id.split('_')[0]
                    print(corp_sents[_id]+' '+corp_texts[_id.split('_')[0]]['url']+'\n')
    
def search_corp(search_str):
    search_res=[]
    for search_inq in search_str.split(' '):
        print('||'+search_inq+'\n')
        for_search =determ_ent(search_inq)
        search(for_search)

        
            
        
    

In [8]:
search_corp('attitudes we')

|attitudes

If we define culture as shared beliefs, values, attitudes and behaviours, it stands to reason then that a person's country of origin impacts their culture. https://www.ielts-exam.net/ielts_writing_samples_task_2/1129/

Another negative result of this change in attitudes is the effect it has on young people all over the world, who spend time in shopping malls. https://www.ielts-exam.net/ielts_writing_samples_task_2/1056/

Another negative result of this change in attitudes is the effect it has on young people all over the world, who spend time in shopping malls. https://www.ielts-exam.net/ielts_writing_samples_task_2/1015/

Therefore, I firmly believe that while every country should be accountable for their emissions and contribute to combating climate change, developed countries have a slightly bigger responsibility to take action than developing countries, as they have both the means and the reasons to do so, as well as the ability to create a formidable wave of change in 